In [1]:
# Necesary imports
import numpy as np
import pandas as pd
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
import random
from statistics import mean 
from ensemble_counterfactuals.common_funcs import train_models
from ensemble_counterfactuals.algorithms import ga, eda, moeda, nsga2, ebna, moebna

C:\Users\Pollo\AppData\Local\Temp\ipykernel_15796\1478943656.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Importing a dataset from uci machine learning repository as an example
nursery = fetch_ucirepo(id=19) 

X = nursery.data.features 
y = nursery.data.targets 

X['class'] = y['class']
class_values = y['class'].unique()
one_sol = False
obj_class = 'spec_prior'

discrete_variables = [True,True,True,True,True,True]
#discrete_variables = None

In [3]:
# Train and test split
train_df, test_df = train_test_split(X, test_size=0.1,random_state=188574)

In [4]:
len(test_df)

173

In [5]:
print(test_df.values[2])

['high' 'vhigh' '5more' 'more' 'small' 'low' 'unacc']


In [6]:
# Train the models with the train and test
train_models(train_df,test_df)

In [7]:
# Execution example
# Change the instance number, objective class and the line after the comment
instance_number = 2
obj_class = "acc"
results = []
dfs = []
times = []
print(f'Instance number: {instance_number}')
cls_val = np.delete(class_values, np.where(class_values == test_df.values[instance_number][-1]))
print(test_df.values[instance_number])
# Change eda.ensemble_coiunter_eda with each possible algorithm, all have the same attributes
df, rem_models, accuracy,time = eda.ensemble_counter_eda(X=train_df,input=test_df.values[instance_number],obj_class=obj_class,test=test_df,discrete_variables=discrete_variables,verbose=False,no_train=True)
if df is not None:
    times.append(time)
    dfs.append(df)
    models = {}
    chg_varaibles = {}
    prediction = {}
    plausible = {}
    for e in df["model"].unique():
        if e != "input":
            models[e] = []
            if e != "baseline":
                chg_varaibles[e] = []
                prediction[e] = []
                plausible[e] = []

    for row in df.values:
        if row[0] == "input":
            continue
        models[row[0]].append(float(row[-2]))
        if row[0] == "baseline":
            continue
        chg_varaibles[row[0]].append(float(row[-4]))   
        prediction[row[0]].append(float(row[-3]))
        plausible[row[0]].append(float(row[-1]))

    baseline = models['baseline'][0]
    model_distances = {}
    for model_name in models.keys():
        if model_name == "input" or model_name == "baseline":
            continue
        dict_dist = {"distance":1000000000000,"baseline":None,'distance_ind':0}
        for ind, elem in enumerate(models[model_name]):
            if dict_dist["distance"] >= elem:
                dict_dist["distance"] = elem
                dict_dist["distance_ind"] = ind
            if elem == baseline:
                dict_dist["baseline"] = elem
                dict_dist["baseline_ind"] = ind
        dict_dist["chg_variables"] = chg_varaibles[model_name][dict_dist["distance_ind"]]
        dict_dist["prediction"] = prediction[model_name][dict_dist["distance_ind"]]    
        dict_dist["plausible"] = plausible[model_name][dict_dist["distance_ind"]] 
        dict_dist["accuracy"] = accuracy[rem_models.index(model_name)]
        model_distances[model_name] = dict_dist
    results.append(model_distances)
print(f'Media tiempos: {mean(times)}')

Instance number: 2
['high' 'vhigh' '5more' 'more' 'small' 'low' 'unacc']
Modelo: nb Sol: unacc 
Modelo: tn Sol: unacc 
Modelo: fssj Sol: unacc 
Modelo: kdb Sol: unacc 
Modelo: tanhc Sol: unacc 
Modelo: nb Sol: 0.8323699 
Modelo: tn Sol: 0.9248555 
Modelo: fssj Sol: 0.6936416 
Modelo: kdb Sol: 0.9595376 
Modelo: tanhc Sol: 0.9421965 
['nb', 'tn', 'fssj', 'kdb', 'tanhc']
	NFEVALS = 300 F = 0.13888888888888887
	X = [1 3 1 2 2 0 3]
	NFEVALS = 260 F = 0.13888888888888887
	X = [1 3 1 2 2 0 3]
	NFEVALS = 300 F = 100.0
	X = [0 3 1 2 2 1 3]
	NFEVALS = 300 F = 0.19444444444444442
	X = [0 2 0 2 2 0 1]
	NFEVALS = 360 F = 0.13888888888888887
	X = [0 2 1 2 2 0 0]
Media tiempos: 1.7109545707702636


In [8]:
# Dictionary with all the results
print(results)

[{'nb': {'distance': 0.13888888888888887, 'baseline': 0.13888888888888887, 'distance_ind': 0, 'baseline_ind': 0, 'chg_variables': 2.0, 'prediction': 0.2188024580848269, 'plausible': 0.047619047619047616, 'accuracy': 0.8323699421965318}, 'tn': {'distance': 0.13888888888888887, 'baseline': 0.13888888888888887, 'distance_ind': 0, 'baseline_ind': 0, 'chg_variables': 2.0, 'prediction': 0.2188024580848269, 'plausible': 0.047619047619047616, 'accuracy': 0.9248554913294798}, 'fssj': {'distance': 100.0, 'baseline': None, 'distance_ind': 0, 'chg_variables': 0.0, 'prediction': 0.9999999778640568, 'plausible': 0.11904761904761904, 'accuracy': 0.6936416184971098}, 'kdb': {'distance': 0.19444444444444442, 'baseline': None, 'distance_ind': 0, 'chg_variables': 3.0, 'prediction': 0.2374806839909006, 'plausible': 0.0, 'accuracy': 0.9595375722543352}, 'tanhc': {'distance': 0.13888888888888887, 'baseline': 0.13888888888888887, 'distance_ind': 0, 'baseline_ind': 0, 'chg_variables': 2.0, 'prediction': 0.229

In [9]:
# Getting the result dataframe with all models
print(obj_class)
display(df)

acc


,model,buying,maint,doors,persons,lug_boot,safety,Chg Variables,Prediction score,Distance,Plausible
0,input,high,vhigh,5more,more,small,low,-,-,-,-
1,nb,med,-,-,-,-,high,2,0.218802,0.138889,0.047619
2,tn,med,-,-,-,-,high,2,0.218802,0.138889,0.047619
3,fssj,-,-,-,-,-,-,0,1.0,100.0,0.119048
4,kdb,-,med,4,-,-,high,3,0.237481,0.194444,0.0
5,tanhc,-,med,-,-,-,high,2,0.229274,0.138889,0.0
6,baseline,-,med,-,-,-,high,2,-,0.138889,-
